In [1]:
import pandas as pd
import re

In [2]:
bici_19 = pd.read_csv('AB_19.csv')
bici_18 = pd.read_csv('AB_18.csv')
bici_17 = pd.read_csv('AB_17.csv')

In [3]:
bici = pd.concat([bici_19, bici_18, bici_17], ignore_index = True)

In [4]:
bici.head()

,fecha,meteo,tipo_accidente,lesividad,distrito,direccion
0,2019-01-01 14:00:00,Despejado,Colision,Moderada,SALAMANCA,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ
1,2019-01-02 21:00:00,Despejado,Colision,Grave,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES
2,2019-01-03 14:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,CALL. FELIPE ALVAREZ 10
3,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA
4,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA


In [5]:
def horario(df):
    df['fecha'] = pd.to_datetime(df['fecha'], yearfirst=True)
    df['hora'] = df.fecha.dt.hour
    df['horario'] = df.hora.apply(lambda x: 'Mañana' if (x>=6 and x<=12) else ('Tarde' if (x>12 and x<=20) else 'Noche'))
    df.drop(['hora'], axis = 1, inplace = True)
    df['dia'] = df.fecha.dt.date
    df.dia = pd.to_datetime(df.dia, yearfirst=True)
    return df


In [6]:
bici = horario(bici)

In [7]:
bici.head()

,fecha,meteo,tipo_accidente,lesividad,distrito,direccion,horario,dia
0,2019-01-01 14:00:00,Despejado,Colision,Moderada,SALAMANCA,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ,Tarde,2019-01-01
1,2019-01-02 21:00:00,Despejado,Colision,Grave,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES,Noche,2019-01-02
2,2019-01-03 14:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,CALL. FELIPE ALVAREZ 10,Tarde,2019-01-03
3,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA,Tarde,2019-01-03
4,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA,Tarde,2019-01-03


In [8]:
bici.shape

(2235, 8)

In [9]:
bici.dtypes

fecha             datetime64[ns]
meteo                     object
tipo_accidente            object
lesividad                 object
distrito                  object
direccion                 object
horario                   object
dia               datetime64[ns]
dtype: object

In [10]:
cal = pd.read_csv ('../raw_data/bicicletas/calendario.csv', encoding='latin_1',sep = ';')

In [11]:
cal.drop(['Dia_semana', 'Tipo de Festivo', 'Festividad'], axis = 1, inplace = True)

In [12]:
cal.Dia = pd.to_datetime(cal['Dia'], dayfirst=True)

In [13]:
cal.rename(columns={'Dia':'dia', 'laborable / festivo / domingo festivo':'festividad'}, inplace=True)

In [14]:
cal.festividad = cal.festividad.apply(lambda x: 'Festivo' if (x=='domingo' or x=='festivo') else 'Laborable')

In [15]:
cal.dtypes

dia           datetime64[ns]
festividad            object
dtype: object

In [16]:
cal.head()

,dia,festividad
0,2013-01-01,Festivo
1,2013-01-02,Laborable
2,2013-01-03,Laborable
3,2013-01-04,Laborable
4,2013-01-05,Laborable


In [17]:
bici = bici.merge(cal, how='left', on='dia')

In [18]:
bici.head()

,fecha,meteo,tipo_accidente,lesividad,distrito,direccion,horario,dia,festividad
0,2019-01-01 14:00:00,Despejado,Colision,Moderada,SALAMANCA,CALL. CASTELLO / CALL. DON RAMON DE LA CRUZ,Tarde,2019-01-01,Festivo
1,2019-01-02 21:00:00,Despejado,Colision,Grave,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES,Noche,2019-01-02,Laborable
2,2019-01-03 14:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,CALL. FELIPE ALVAREZ 10,Tarde,2019-01-03,Laborable
3,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA,Tarde,2019-01-03,Laborable
4,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALL. REAL DE ARGANDA,Tarde,2019-01-03,Laborable


In [19]:
bici.direccion = bici.direccion.apply(lambda x: re.sub(r'CALL.', 'CALLE', x))

In [20]:
bici.direccion = bici.direccion + ' Madrid'

In [21]:
bici['lon']=''
bici['lat']=''

In [22]:
from googlemaps import Client as GoogleMaps
from dotenv import load_dotenv
import os

In [23]:
load_dotenv()
G_KEY = os.getenv("KEY_GOOGLE")

In [24]:
gmaps=GoogleMaps(G_KEY)

In [25]:
addresses = bici[0:3]

In [26]:
addresses

,fecha,meteo,tipo_accidente,lesividad,distrito,direccion,horario,dia,festividad,lon,lat
0,2019-01-01 14:00:00,Despejado,Colision,Moderada,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,Tarde,2019-01-01,Festivo,,
1,2019-01-02 21:00:00,Despejado,Colision,Grave,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,Noche,2019-01-02,Laborable,,
2,2019-01-03 14:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,CALLE FELIPE ALVAREZ 10 Madrid,Tarde,2019-01-03,Laborable,,


In [33]:
addresses.iat[i,addresses.columns.get_loc('direccion')]

'AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALES Madrid'

In [28]:
# for x in range(len(addresses)):
#     geocode_result = gmaps.geocode(addresses['direccion'][x])
#     addresses['lat'][x] = geocode_result[0]['geometry']['location'] ['lat']
#     addresses['lon'][x] = geocode_result[0]['geometry']['location']['lng']

/home/ovi/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ovi/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [25]:
wasd = bici[0:2]

In [26]:
wasd

,fecha,meteo,tipo_accidente,lesividad,distrito,direccion,horario,dia,festividad,lon,lat
0,2019-01-01 14:00:00,Despejado,Colision,Moderada,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,Tarde,2019-01-01,Festivo,,
1,2019-01-02 21:00:00,Despejado,Colision,Grave,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,Noche,2019-01-02,Laborable,,


In [27]:
for i in range(len(wasd)):
    geocode_result = gmaps.geocode(wasd.iat[i, wasd.columns.get_loc('direccion')])
    try:
        lat = geocode_result[0]["geometry"]["location"]["lat"]
        lon = geocode_result[0]["geometry"]["location"]["lng"]
        wasd.iat[i, wasd.columns.get_loc('lat')] = lat
        wasd.iat[i, wasd.columns.get_loc('lon')] = lon
    except:
        lat = None
        lon = None

In [28]:
wasd 

,fecha,meteo,tipo_accidente,lesividad,distrito,direccion,horario,dia,festividad,lon,lat
0,2019-01-01 14:00:00,Despejado,Colision,Moderada,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,Tarde,2019-01-01,Festivo,-3.68121,40.429
1,2019-01-02 21:00:00,Despejado,Colision,Grave,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,Noche,2019-01-02,Laborable,-3.65453,40.4669


In [24]:
bici.head()

,fecha,meteo,tipo_accidente,lesividad,distrito,direccion,horario,dia,festividad,lon,lat
0,2019-01-01 14:00:00,Despejado,Colision,Moderada,SALAMANCA,CALLE CASTELLO / CALLE DON RAMON DE LA CRUZ Ma...,Tarde,2019-01-01,Festivo,,
1,2019-01-02 21:00:00,Despejado,Colision,Grave,HORTALEZA,AVDA. GRAN VIA DE HORTALEZA / GTA. LUIS ROSALE...,Noche,2019-01-02,Laborable,,
2,2019-01-03 14:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,CALLE FELIPE ALVAREZ 10 Madrid,Tarde,2019-01-03,Laborable,,
3,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,Tarde,2019-01-03,Laborable,,
4,2019-01-03 13:00:00,Despejado,Colision,Leve,VILLA DE VALLECAS,AVDA. SANTA EUGENIA / CALLE REAL DE ARGANDA Ma...,Tarde,2019-01-03,Laborable,,


In [15]:
test = pd.DataFrame({
    'Day': [1,2,3,4,5,6,7,8,9],
    'Fest': ['A','B','B','B','A','A','A','B','A']
})

In [16]:

for i in range (1, test.shape[0]):
    if test.Fest[i]=='B' and test.Fest[i+1]=='A':
        test['NEW'][i]='C'
    else:
        test['NEW']=test.Fest
        

/home/ovi/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
test

,Day,Fest,NEW
0,1,A,A
1,2,B,B
2,3,B,B
3,4,B,B
4,5,A,A
5,6,A,A
6,7,A,A
7,8,B,B
8,9,A,A
